## Functions used for atn_resilience analysis.

In [1]:
#imports for the notebook
import os
dir_path = os.getcwd()
script_dir = dir_path + '/atnresilience'
os.chdir(script_dir)
import atn_analysis
import atn_tools as oc
import pandas as pd

In [2]:
#Create the paths for the database and raw files
#dir_path is the path to the main directory

db_path = '%s/data/processed/atn_db.sqlite' %(dir_path,)
raw_path = '%s/data/raw/'%(dir_path,)
processed_direc = '%s/data/processed/'%(dir_path,)
graph_direc = '%s/data/graph/'%(dir_path,)
print(db_path)
print(processed_direc)

/Users/allen/Documents/atnresilience/data/processed/atn_db.sqlite
/Users/allen/Documents/atnresilience/data/processed/


In [3]:
start_year = 2015
end_year = 2017
airline_list = ['ALL','AA','DL','UA','WN']
include_data = 'ADM'
year_list = list(range(start_year, end_year+1))

## Calculate Mahalanobis Data
Run the DB query that will also calculate the Mahalanobis Distance for that data set. Data for the years queried must be imported to the database beforehand. The M-D data will be saved in the /processed folder.

For the include_data parameter, the following options are possible:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; CC: Cancellations only<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ADD: Arrival delays including diversions<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ADM: Purely arrival delays excluding cancellations or diversions<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; DCC: Combined delay. If arrival delay is greater than a set threshold, the flight is considered cancelled<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; DD: Departure delays. Does not include cancelled or diverted flights<br>

In [5]:
#Specify year ranges to analyze
MD_min_year = 2015
MD_max_year = 2017

#Specify the data dropping method
md_include_data = 'ADM'

#Specify a list of airlines to analyze. To do all airlines, put ''
md_airlines = ['ALL','AA','DL','UA','WN']

In [5]:
#Run the MD query and analysis
# MD_year_range =  list(range(MD_min_year, MD_max_year+1))

for airline in airline_list:
    for year in year_list:
        MD = atn_analysis.MahalanobisDistance(year, airline)
        MD.mahalanobis_distance()
    print('Finished M-Distance calculation for %s.'%(airline,))

2015 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/ALL_2015_MDdata_ADM.csv
2016 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/ALL_2016_MDdata_ADM.csv
2017 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/ALL_2017_MDdata_ADM.csv
Finished M-Distance calculation for ALL.
2015 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/AA_2015_MDdata_ADM.csv
2016 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/AA_2016_MDdata_ADM.csv
2017 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/AA_2017_MDdata_ADM.csv
Finished M-Distance calculation for AA.
2015 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/DL_2015_MDdata_ADM.csv
2016 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/DL_2016_MDdata_ADM.csv
2017 M-D Data created at /Users/allen/Documents/atnresilience/data/processed/DL_2017_MDdata_ADM.csv
Finished M-Dista

## Create Z-Score Data
Run the DB query that will also calculate the Z-score for that data set. Data for the years queried must be imported to the database beforehand. The z-score data will be saved in the /processed folder.

For the include_data parameter, the following options are possible:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; CC: Cancellations only<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ADD: Arrival delays including diversions<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ADM: Purely arrival delays excluding cancellations or diversions<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; DCC: Combined delay. If arrival delay is greater than a set threshold, the flight is considered cancelled<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; DD: Departure delays. Does not include cancelled or diverted flights<br>

In [3]:
#Specify year ranges to analyze
z_min_year = 2015
z_max_year = 2015

#Specify the data dropping method
z_include_data = 'ADM'

#Specify list of airlines to analyze. To do all airlines, put ''
z_airlines = ['AA']

In [4]:
#Run the Z-score query and analysis
z_year_range =  list(range(z_min_year, z_max_year+1))

for airline in z_airlines:
    for i in z_year_range:
        atn_analysis.z_score(db_path, i, airline, z_include_data, processed_direc)
    print('Finished z-score Calculation for %s for all specified years' %(airline,))

Airport data imported from Query
Calculating metrics
['ABQ' 'ATL' 'AUS' 'BNA' 'BWI' 'CLT' 'DCA' 'DEN' 'DFW' 'DTW' 'ELP' 'FLL'
 'HNL' 'IAD' 'IAH' 'IND' 'JAX' 'LAS' 'LAX' 'LIH' 'MCI' 'MCO' 'MIA' 'MSP'
 'MSY' 'OGG' 'OMA' 'ONT' 'ORD' 'PBI' 'PDX' 'PHL' 'PHX' 'PSP' 'RDU' 'RNO'
 'RSW' 'SAN' 'SAT' 'SEA' 'SFO' 'SJC' 'SJU' 'SLC' 'SNA' 'STL' 'TPA' 'TUL'
 'TUS']
Finished z-score Calculation for AA for all specified years


## Create raw daily delay data for each specified airline

In [3]:
#Specify year ranges to analyze
delay_min_year = 2015
delay_max_year = 2017

#Specify the data dropping method
delay_include_data = 'ADM'

#Specify a list of airlines to analyze. To do all airlines, put ''
delay_airlines = ['ALL','AA','DL','UA','WN']

In [9]:
delay_year_range =  list(range(delay_min_year, delay_max_year+1))
queries = atn_analysis.AnalysisQueries()
for airline in delay_airlines:
    for year in delay_year_range:
        init_df = queries.airline_query(year, airline, delay_include_data)
        init_df.rename(columns={'Day':'Day_of_Year'},inplace=True)
        daily_mean = init_df.groupby('Day_of_Year',as_index=False)['Pos_Arr_Delay'].mean()
        file_name = '%s%s_%s_daily_delay.csv' %(processed_direc,airline,year)
        daily_mean.to_csv(file_name, index=False)

6-01-01            1                      TPA       7.789474
...           ...          ...                      ...            ...
10950  2016-12-31          366                      ATL       0.000000
10951  2016-12-31          366                      AUS       2.000000
10952  2016-12-31          366                      BOS       3.625000
10953  2016-12-31          366                      CLE       3.200000
10954  2016-12-31          366                      DEN       4.563910
10955  2016-12-31          366                      DFW       0.000000
10956  2016-12-31          366                      DTW       0.000000
10957  2016-12-31          366                      HNL      46.555556
10958  2016-12-31          366                      IAH       5.362319
10959  2016-12-31          366                      KOA       6.333333
10960  2016-12-31          366                      LAS       4.777778
10961  2016-12-31          366                      LAX       5.378788
10962  2016-12-3

## Generate metrics on dropped flight data for each specified airline
Saves as a single file ordered by airline and year

In [3]:
dropped_years = [2015,2016,2017]
dropped_airlines = ['','AA','DL','UA','WN']

In [4]:
all_metrics = []
cols = ['Year','Airline','Airport % Dropped','Airport Total','Airport Kept','Flights Total','Flights Dropped','Flights % Dropped']
for airline in dropped_airlines:
    for year in dropped_years:
        all_metrics.append(oc.dropped_metrics(db_path,year,airline,'ADM'))

metrics_df = pd.DataFrame(all_metrics, columns=cols)
dropped_path = processed_direc + 'dropped_metrics.csv'
metrics_df.to_csv(dropped_path)